# Part III: Model Development

Key Points:
1. Training dataset will be historical data from 2016-2021
2. Test dataset (for now) will be data from 2022 season
3. Models to Try:
   - Logistic Regression
   - K-Nearest Neighbors
   - Naive Bayes Classifier
   - LightGBM and XGBoost

---

### 1) Imports

In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install import-ipynb

In [3]:
import III_feature_engineering as III

In [4]:
from importlib import reload
reload(III);

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


#import lightgbm as lgb
#from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier as RF

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split as tts

from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Load Cleaned Data produced from Notebook II
nba_clean_data = pd.read_csv('../data/cleaned_nba_data.csv')

In [10]:
nba_clean_data

,team,team_opp,season,date,home,fg,fga,fg%,3p,3pa,...,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp,won,target,game_id
0,DET,ATL,2016,2015-10-27,0,37.0,96.0,0.385,12.0,29.0,...,40.4,59.5,9.4,6.0,14.5,98.6,111.2,True,1,1
1,ATL,DET,2016,2015-10-27,1,37.0,82.0,0.451,8.0,27.0,...,59.6,62.2,5.2,5.5,12.3,111.2,98.6,False,1,2
2,NOP,GSW,2016,2015-10-27,0,35.0,83.0,0.422,6.0,18.0,...,62.9,70.7,8.0,10.8,15.9,110.9,94.9,False,0,3
3,GSW,NOP,2016,2015-10-27,1,41.0,96.0,0.427,9.0,30.0,...,37.1,60.0,9.0,4.5,15.9,94.9,110.9,True,1,4
4,CLE,CHI,2016,2015-10-27,0,38.0,94.0,0.404,9.0,29.0,...,48.5,35.1,6.0,15.4,11.8,97.5,95.5,False,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20407,DEN,MIA,2023,2023-06-07,0,41.0,80.0,0.513,5.0,18.0,...,36.3,58.8,7.7,4.8,3.8,103.8,120.4,True,1,20408
20408,DEN,MIA,2023,2023-06-09,0,39.0,79.0,0.494,14.0,28.0,...,52.1,65.7,2.3,5.9,13.9,107.2,121.9,True,1,20409
20409,MIA,DEN,2023,2023-06-09,1,35.0,78.0,0.449,8.0,25.0,...,47.9,66.7,12.4,13.2,6.4,121.9,107.2,False,0,20410
20410,DEN,MIA,2023,2023-06-12,1,38.0,84.0,0.452,5.0,28.0,...,43.6,54.5,9.4,12.5,7.2,92.6,97.8,True,-1,20411


In [22]:
sample_data = III.execute_feature_engineering(nba_clean_data, averages=True)
sample_data = sample_data[sample_data['team'] == 'DET']
sample_data

,team,team_opp,season,date,home,won,target,game_id,fg,fga,...,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,ortg_opp,drtg_opp
0,DET,ATL,2016,2015-10-27,0,True,1,1,37.000,96.000,...,0.183,16.300,58.900,40.400,59.500,9.400,6.000,14.500,98.600,111.200
29,DET,UTA,2016,2015-10-28,1,True,1,30,33.500,85.000,...,0.280,10.300,81.000,46.900,42.900,4.500,9.100,12.500,96.900,102.400
49,DET,CHI,2016,2015-10-30,1,True,0,50,35.333,91.333,...,0.277,15.400,71.367,46.633,45.200,7.567,9.833,15.467,100.833,96.400
111,DET,IND,2016,2015-11-03,1,False,1,112,34.750,90.000,...,0.276,21.775,68.375,44.900,56.275,11.350,7.075,16.750,96.225,102.675
152,DET,PHO,2016,2015-11-06,0,True,1,153,35.200,86.800,...,0.279,24.617,84.350,53.367,48.783,9.500,7.083,13.400,102.000,98.683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20107,DET,ORL,2023,2023-04-02,0,False,0,20108,39.641,87.282,...,0.297,23.421,77.532,50.563,57.091,7.329,9.306,12.785,112.894,114.846
20140,DET,MIA,2023,2023-04-04,1,False,0,20141,39.595,87.101,...,0.278,22.604,77.776,49.097,60.548,8.347,6.256,11.765,113.686,114.582
20154,DET,BRK,2023,2023-04-05,1,False,1,20155,39.612,87.062,...,0.260,19.244,74.066,47.409,61.511,7.226,11.074,11.920,116.406,115.253
20190,DET,IND,2023,2023-04-07,0,True,0,20191,39.667,87.099,...,0.266,23.163,72.612,47.905,64.243,7.616,10.364,12.353,115.502,118.757


In [ ]:
# add new change: 12/2/24 test change for Git

---
### 2) Create Training and Testing Sets

In [76]:

def train_test_split(data,
                     model_vars,
                     year,
                     num_games_list,
                     remove_placeholder_target = False,
                     compute_avg = False,
                     compute_rolling_avg = False,
                     drop_na_rows=False
                    ):
    '''
    Return the training and testing features and targets for the given nba dataset:

    original_data = DataFrame with all NBA Box Score data from 2016-2022
    transformed_data = DataFrame with transformed features, such as rolling averages, W/L streaks

    '''
    if remove_placeholder_target:
        target_indices_drop = data[data['target'] == -1].index # Remove rows where target = -1 (i.e. last game of the season), so no outcome
        new_data = data.drop(target_indices_drop)
    
    
    # Subset training and testing data by year
    new_data = new_data.sort_values(['team','season','date']).reset_index(drop=True)
    new_data = new_data.sort_values(['season','date']).reset_index(drop=True) # Weird sorting to make sure train and test targets match up

    nba_train = new_data[new_data['season'] != year].reset_index(drop=True)
    nba_test = new_data[new_data['season'] == year].reset_index(drop=True) # OOT testing

    train_target = nba_train['target'] # Subset the targets before potentially computing rolling_averages
    test_target = nba_test['target']

    if compute_avg:
        train_data = III.create_averages(nba_train,
                                         model_vars,
                                         train_only=True,
                                         include_target=False
                                        )
        test_data = III.create_averages(nba_test,
                                        model_vars,
                                        train_only=True,
                                        include_target=False)
    
    elif compute_rolling_avg:
        train_data = III.create_rolling_averages(nba_train,
                                             model_vars,
                                             num_games_list,
                                             train_only=True
                                            ) 
        test_data = III.create_rolling_averages(nba_test,
                                           model_vars,
                                           num_games_list,
                                           train_only=True
                                           )

        if drop_na_rows: # For certain algorithms that don't recognize NaN values (i.e. first 5 rows of each team)
            nan_train_idx = train_data.index[train_data.isna().any(axis=1)].to_list()
            nan_test_idx = test_data.index[test_data.isna().any(axis=1)].to_list()
            
            train_data = train_data.drop(nan_train_idx).reset_index(drop=True)
            test_data = test_data.drop(nan_test_idx).reset_index(drop=True)

            # Need to drop the same row indices for target
            train_target = train_target.drop(nan_train_idx).reset_index(drop=True)
            test_target = test_target.drop(nan_test_idx).reset_index(drop=True)
            
    else:
        train_data = nba_train[model_vars]
        test_data = nba_test[model_vars]
    
    return train_data, train_target, test_data, test_target

### 2b) Establish Training and Testing Data

In [18]:
# Make sure functions work from other .py file
model_variables = III.create_model_variables(nba_clean_data) # Establish model variables
nba_transformed = III.create_rolling_averages(nba_clean_data, # Create new dataframe with two different rolling averages
                                          model_variables,
                                          num_games_list=[3,5],
                                          train_only=False)

In [37]:
# Establish training and testing sets
model_variables = III.create_model_variables(nba_clean_data) # Establish model variables
X_train, y_train, X_test, y_test = train_test_split(nba_clean_data,
                                                    model_variables,
                                                    2022,
                                                    [3,5,10], 
                                                    True,True,True)


### 3) Baseline Logistic Regression Model

1) Trial 1: Baseline (max_iter = 5000)
    - Parameters: 2 rolling averages (3 and 5 games)
    - Train Accuracy = 0.53
    - Test Accuracy = 0.52
    - Precision (Class 1) = 0.51
    - Recall (Class 1) = 0.61
    - F1-Score = 0.56

2) Trial 2: Only 5 game average:
    - Train = 0.52
    - Test = 0.49
    - Precision = 0.49
    - Recall = 0.64
    - F1-Score = 0.55

3) Trial 3: 3,5,10 game averages:
    - Train = 0.54
    - Test = 0.52
    - Precision = 0.52
    - Recall = 0.61
    - F1-Score = 0.56


In [38]:
# Train baseline logistic Regression Model
log_reg = LR(max_iter = 5000, random_state = 0) #baseline model
ss = SS()

train_s = ss.fit_transform(X_train) # Standardize of training and testing features
test_s = ss.transform(X_test) 

log_reg.fit(train_s,y_train) # fit logistic regression

[log_reg.score(train_s,y_train), log_reg.score(test_s,y_test)]

[0.534969233078193, 0.5191815856777494]

In [39]:
train_pred = log_reg.predict(train_s)
test_pred = log_reg.predict(test_s)

print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.52      0.43      0.47      1175
           1       0.52      0.61      0.56      1171

    accuracy                           0.52      2346
   macro avg       0.52      0.52      0.52      2346
weighted avg       0.52      0.52      0.52      2346



---
### 4) Logistic Regression with Consecutive Averages

In [81]:
model_variables = III.create_model_variables(nba_clean_data) # Establish model variables
nba_transformed = III.create_averages(nba_clean_data,model_variables,train_only=True,include_target=False)

X_train, y_train, X_test, y_test = train_test_split(nba_clean_data,
                                                    model_variables,
                                                    2022,
                                                    None, 
                                                    True,True,
                                                    False,False)


In [83]:
log_reg = LR(max_iter = 5000, random_state = 0) #baseline model
ss = SS()

train_s = ss.fit_transform(X_train) # Standardize of training and testing features
test_s = ss.transform(X_test) 

log_reg.fit(train_s,y_train) # fit logistic regression

[log_reg.score(train_s,y_train), log_reg.score(test_s,y_test)]

[0.5102368526696106, 0.5107033639143731]

In [84]:
train_pred = log_reg.predict(train_s)
test_pred = log_reg.predict(test_s)

print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.51      0.55      0.53      1308
           1       0.51      0.47      0.49      1308

    accuracy                           0.51      2616
   macro avg       0.51      0.51      0.51      2616
weighted avg       0.51      0.51      0.51      2616



---
### 5) K-Nearest Neighbors

In [85]:
train_s = ss.fit_transform(X_train) # Standardize of training and testing features
test_s = ss.transform(X_test) 

k = 13
knn = KNN(n_neighbors=k)
knn.fit(train_s,y_train) # fit logistic regression
#print(knn.score(train_s,y_train),knn.score(test_s, y_test))

test_pred = knn.predict(test_s)
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.50      0.52      0.51      1308
           1       0.50      0.48      0.49      1308

    accuracy                           0.50      2616
   macro avg       0.50      0.50      0.50      2616
weighted avg       0.50      0.50      0.50      2616



---
### 6) LightGBM Classiifer

In [88]:
lgb_classifier = lgb.LGBMClassifier(num_leaves=10, max_depth=2,random_state=0,verbose=-1)

lgb_classifier.fit(X_train,y_train)
print([lgb_classifier.score(X_train,y_train),lgb_classifier.score(X_test,y_test)])
test_pred = lgb_classifier.predict(X_test)
print(classification_report(y_test, test_pred))

[0.5501137428074401, 0.518348623853211]
              precision    recall  f1-score   support

           0       0.52      0.55      0.53      1308
           1       0.52      0.49      0.50      1308

    accuracy                           0.52      2616
   macro avg       0.52      0.52      0.52      2616
weighted avg       0.52      0.52      0.52      2616



---
## Part II: Combine Features

Note: Need to make more functions for train test split

In [101]:
model_variables = III.create_model_variables(nba_clean_data) # Establish model variables
nba_transformed = III.create_averages(nba_clean_data,model_variables,train_only=True,include_target=False)

X_train, y_train, X_test, y_test = train_test_split(nba_clean_data,
                                                    model_variables,
                                                    2022,
                                                    [3,5], 
                                                    True,False,
                                                    True,False)


In [102]:
X_train

,fga_avg_L3,fg%_avg_L3,3pa_avg_L3,3p%_avg_L3,fta_avg_L3,ft%_avg_L3,orb_avg_L3,trb_avg_L3,ast_avg_L3,stl_avg_L3,...,tov_opp_avg_L5,pf_opp_avg_L5,pts_opp_avg_L5,ts%_opp_avg_L5,efg%_opp_avg_L5,3par_opp_avg_L5,drb%_opp_avg_L5,trb%_opp_avg_L5,ortg_opp_avg_L5,drtg_opp_avg_L5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82.666667,0.463667,24.666667,0.353667,21.000000,0.755000,7.333333,41.333333,23.666667,9.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84.666667,0.453333,25.333333,0.335333,20.666667,0.798000,8.000000,44.000000,23.666667,9.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87.000000,0.421667,24.333333,0.307333,19.333333,0.825000,11.000000,48.000000,23.333333,9.333333,...,16.6,18.8,97.0,0.4958,0.4580,0.3422,79.74,53.34,100.26,102.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14941,92.000000,0.500000,26.666667,0.322000,22.333333,0.809667,10.000000,51.333333,24.333333,5.333333,...,13.2,18.6,113.6,0.5522,0.5042,0.4342,77.84,48.18,109.36,113.80
14942,91.000000,0.524333,23.000000,0.347667,20.333333,0.806333,9.000000,47.333333,25.666667,5.333333,...,12.6,18.0,114.6,0.5464,0.4998,0.4262,77.12,47.10,110.90,114.76
14943,91.000000,0.513333,23.333333,0.330333,23.666667,0.734000,9.000000,44.333333,27.333333,4.666667,...,11.4,18.0,117.6,0.5584,0.5166,0.3882,76.82,48.00,114.30,110.18
14944,92.000000,0.451667,25.666667,0.240000,23.333333,0.731000,11.000000,40.666667,22.000000,4.000000,...,12.2,20.0,122.0,0.5910,0.5598,0.3682,77.02,50.24,118.90,108.06


In [103]:
X_train_2, y_train_2, X_test_2, y_test_2 = train_test_split(nba_clean_data,
                                                            model_variables,
                                                            2022,
                                                            None, 
                                                            True,True,
                                                            False,False)

In [107]:
train = pd.concat([X_train_2,X_train],axis=1)
test = pd.concat([X_test_2,X_test],axis=1)

In [108]:
lgb_classifier = lgb.LGBMClassifier(num_leaves=10, max_depth=2,random_state=0,verbose=-1)

lgb_classifier.fit(train,y_train)
print([lgb_classifier.score(train,y_train),lgb_classifier.score(test,y_test)])
test_pred = lgb_classifier.predict(test)
print(classification_report(y_test, test_pred))

[0.5885855747357153, 0.5053516819571865]
              precision    recall  f1-score   support

           0       0.51      0.39      0.44      1308
           1       0.50      0.62      0.56      1308

    accuracy                           0.51      2616
   macro avg       0.51      0.51      0.50      2616
weighted avg       0.51      0.51      0.50      2616

